In [ ]:
import pandas as pd
import glob

# Path to your CSV files (assuming they are in the same directory)
file_path_pattern = "2411*.csv"

# Initialize total line count
total_lines = 0

# Count lines in each file and print them
for file in glob.glob(file_path_pattern):
    with open(file, 'r') as f:
        line_count = sum(1 for line in f) - 1  # Exclude header row
        print(f"{file}: {line_count} lines")
        total_lines += line_count

# Read and concatenate all CSV files
all_files = glob.glob(file_path_pattern)
df = pd.concat([pd.read_csv(file) for file in all_files], ignore_index=True)

# Check if merged DataFrame has the correct total lines (including header)
merged_line_count = len(df)
print(f"Merged file line count (excluding header): {merged_line_count}")
print("Line count verification:", "PASS" if merged_line_count == total_lines else "FAIL")

# Save the merged DataFrame to a new CSV file if needed
df.to_csv("merged_investments.csv", index=False)


In [ ]:
import pandas as pd

# Replace 'investments1.csv' with the path to one of your files
df_sample = pd.read_csv('merged_investments.csv')
print(df_sample.columns)


In [ ]:
import pandas as pd
import glob

# Path to your CSV files
file_path_pattern = "merged_investments.csv"

# Columns to drop
columns_to_drop = [
     'Industries'
]

# Read, drop columns, and concatenate all CSV files
all_files = glob.glob(file_path_pattern)
df = pd.concat([pd.read_csv(file).drop(columns=columns_to_drop) for file in all_files], ignore_index=True)

# Save the merged DataFrame to a new CSV file if needed
df.to_csv("merged_investments.csv", index=False)


In [ ]:
# Run the Chroma db 
chroma run &

#sqlite3 
sqlite3 ai_investments.db 



In [ ]:
#understand the table structure to create the db in sqlite3 

In [ ]:
import pandas as pd

# Load the merged CSV file to inspect its structure and columns
# Assuming the file is named 'merged_investments.csv'
file_path = "merged_investments.csv"

try:
    # Load a sample of the merged CSV file to examine its structure
    df_merged = pd.read_csv(file_path)
    # Display the first few rows and column names to understand the data
    df_merged_head = df_merged.head()
    df_merged_columns = df_merged.columns
    df_merged_info = df_merged.info()
except FileNotFoundError:
    df_merged_head = "File not found."
    df_merged_columns = "File not found."
    df_merged_info = "File not found."

df_merged_head, df_merged_columns, df_merged_info


In [ ]:
#create the sqllite3 table 

CREATE TABLE investments (
    "Organization Name" TEXT,
    "Last Funding Type" TEXT,
    "Last Funding Date" TEXT,
    "Last Funding Amount (in USD)" INTEGER,
    "Headquarters Location" TEXT,
    "Description" TEXT,
    "Top 5 Investors" TEXT,
    "Founders" TEXT,
    "Most Recent Valuation Range" TEXT,
    "Founded Date" TEXT,
    "Twitter" TEXT,
    "LinkedIn" TEXT,
    "Full Description" TEXT,
    "Website" TEXT
);


sqlite> .tables
investments
sqlite> .schema investments
CREATE TABLE investments (
    "Organization Name" TEXT,
    "Last Funding Type" TEXT,
    "Last Funding Date" TEXT,
    "Last Funding Amount (in USD)" INTEGER,
    "Headquarters Location" TEXT,
    "Description" TEXT,
    "Top 5 Investors" TEXT,
    "Founders" TEXT,
    "Most Recent Valuation Range" TEXT,
    "Founded Date" TEXT,
    "Twitter" TEXT,
    "LinkedIn" TEXT,
    "Full Description" TEXT,
    "Website" TEXT
);
sqlite>


#now install the .csv into the table 
.mode csv
.import merged_investments.csv investments

sqlite> .mode csv
sqlite> .import merged_investments.csv investments
sqlite> SELECT * FROM investments LIMIT 5;
.exit 

### First check if there is any db in chromadb

In [ ]:
from chromadb import Client

# Initialize the ChromaDB client
client = Client()

# List all existing collections
collections = client.list_collections()

# Check if there are any collections
if collections:
    print("Existing collections:")
    for collection in collections:
        print(f"- Collection name: {collection.name}, Number of items: {collection.count()}")
else:
    print("No collections found in ChromaDB.")


### Install embeddings to chromadb along with organization name metadata
out of these columns in the data we will only keep the "Organisation Name" as it is and take the embeddings of "Description" and "Full Description" and add them to chroma 

In [ ]:
# first check if there are any collections in the chromadb

from chromadb import Client

# Initialize ChromaDB client
client = Client()

# List all existing collections
collections = client.list_collections()

# Check if there are any collections and display their details
if collections:
    print("Existing collections in ChromaDB:")
    for collection in collections:
        collection_name = collection.name
        item_count = collection.count()  # Count the number of items in the collection
        print(f"- Collection name: {collection_name}, Number of items: {item_count}")
else:
    print("No collections found in ChromaDB. Database is empty.")


In [ ]:
import os
import pandas as pd
import logging
from openai import AzureOpenAI
import chromadb

# Set up logging for every 100 embeddings processed
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Suppress detailed logs from the openai library
logging.getLogger("openai").setLevel(logging.ERROR)

# Initialize AzureOpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT")
)

# Function to get embeddings with a check for NaN
def get_embedding(text, model="text-embedding-3-small-eastus"):
    if isinstance(text, float):  # Check if the text is NaN
        return None
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Load the merged CSV file
df = pd.read_csv("merged_investments.csv")

# Initialize columns for embeddings
df['description_embedding'] = None
df['full_description_embedding'] = None

# Generate embeddings for both "Description" and "Full Description", with logging every 100 rows
for i in range(len(df)):
    # Generate embeddings for "Description" and "Full Description" for each row
    df.at[i, 'description_embedding'] = get_embedding(df.at[i, 'Description'])
    df.at[i, 'full_description_embedding'] = get_embedding(df.at[i, 'Full Description'])

    # Log progress every 100 rows
    if (i + 1) % 100 == 0:
        logging.info(f"Processed {i + 1} embeddings out of {len(df)} rows")

# Drop rows where either embedding is missing
df = df.dropna(subset=['description_embedding', 'full_description_embedding'])

# Combine the embeddings (e.g., concatenate or average; here we concatenate)
df['combined_embedding'] = [desc + full_desc for desc, full_desc in zip(df['description_embedding'], df['full_description_embedding'])]

# Initialize ChromaDB client and create collection
chroma_client = chromadb.Client()
collection = chroma_client.create_collection("ai_investments")

# Add records to ChromaDB
collection.add(
    documents=df['Description'].tolist(),  # Use descriptions as documents
    embeddings=df['combined_embedding'].tolist(),  # Store combined embeddings
    metadatas=[{'Organization Name': name} for name in df['Organization Name']],  # Only "Organization Name" as metadata
    ids=[f"id_{i}" for i in range(len(df))]  # Unique ids for each entry
)


### ok next, lets'check if the embeddings are now stored in the chromadb and make a query to it 

In [ ]:
# Fetch sample data from the collection without using embeddings
sample_results = collection.get(
    limit=5  # Adjust the number as needed to retrieve a sample of items
)

# Display sample items
for i, metadata in enumerate(sample_results['metadatas']):
    print(f"Item {i + 1}:")
    print("Organization Name:", metadata.get("Organization Name"))
    print("Document:", sample_results['documents'][i])
    print("\n---\n")


### ok now let's do a similarity query 

In [ ]:
import os
import logging
from openai import AzureOpenAI
from chromadb import Client

# Set up logging for minimal output
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.getLogger("openai").setLevel(logging.ERROR)

# Initialize AzureOpenAI client for generating embeddings
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT")
)

# Function to generate embeddings for text queries
def get_embedding(text, model="text-embedding-3-small-eastus"):
    if isinstance(text, float):  # Check if the text is NaN
        return None
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Initialize the ChromaDB client and get the collection
chroma_client = Client()
collection = chroma_client.get_collection("ai_investments")  # Ensure this matches your collection name

# Function to search for similar startups using query embeddings
def search_similar_startups(query_text, top_n=5):
    # Generate two embeddings for the query text and concatenate them to match the 3072-dimensional format
    description_embedding = get_embedding(query_text)
    full_description_embedding = get_embedding(query_text)
    
    if description_embedding is None or full_description_embedding is None:
        print("Failed to generate embeddings for the query text.")
        return []
    
    # Concatenate the two embeddings
    query_embedding = description_embedding + full_description_embedding  # Now 3072 dimensions
    
    # Query the ChromaDB collection with the concatenated embedding
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_n,
        include=["metadatas", "documents", "distances"]  # Retrieve metadata, documents, and distances
    )
    
    # Flatten metadata and distances if necessary
    flattened_metadatas = [item for sublist in results['metadatas'] for item in sublist]
    distances = results.get('distances', [])
    flattened_distances = [distance for sublist in distances for distance in sublist] if isinstance(distances[0], list) else distances
    
    # Extract startup names and similarity scores
    startup_results = [
        {"startup_name": result.get("Organization Name"), "similarity_score": 1 - distance}  # Convert distance to similarity score
        for result, distance in zip(flattened_metadatas, flattened_distances)
        if "Organization Name" in result
    ]
    
    return startup_results


# Example usage: Search for similar startups based on a query
query = "ecommerce"
similar_startups = search_similar_startups(query, top_n=100)

# Print the results with similarity scores
for startup in similar_startups:
    print(f"Startup: {startup['startup_name']}, Similarity Score: {startup['similarity_score']}")


In [ ]:
#Query sqlite3 e.g. 
sqlite> SELECT * FROM investments WHERE "Organization Name" = 'PropulsionAI';

In [ ]:
import os
import sqlite3
import csv
import logging
from openai import AzureOpenAI
from chromadb import Client

# Set up logging for minimal output
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.getLogger("openai").setLevel(logging.ERROR)

# Initialize AzureOpenAI client for generating embeddings
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT")
)

# Function to generate embeddings for text queries
def get_embedding(text, model="text-embedding-3-small-eastus"):
    if isinstance(text, float):  # Check if the text is NaN
        return None
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Initialize the ChromaDB client and get the collection
chroma_client = Client()
collection = chroma_client.get_collection("ai_investments")  # Ensure this matches your collection name

# Function to search for similar startups using concatenated query embeddings
def search_similar_startups(query_text, top_n=100):
    # Generate two embeddings for the query text and concatenate them to match the 3072-dimensional format
    description_embedding = get_embedding(query_text)
    full_description_embedding = get_embedding(query_text)
    
    if description_embedding is None or full_description_embedding is None:
        print("Failed to generate embeddings for the query text.")
        return []
    
    # Concatenate the two embeddings to form a 3072-dimensional vector
    query_embedding = description_embedding + full_description_embedding  # Now 3072 dimensions
    
    # Query the ChromaDB collection with the concatenated embedding
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_n,
        include=["metadatas", "documents", "distances"]  # Retrieve metadata, documents, and distances
    )
    
    # Flatten metadata and distances if necessary
    flattened_metadatas = [item for sublist in results['metadatas'] for item in sublist]
    distances = results.get('distances', [])
    flattened_distances = [distance for sublist in distances for distance in sublist] if isinstance(distances[0], list) else distances
    
    # Extract startup names and similarity scores
    startup_results = [
        {"startup_name": result.get("Organization Name"), "similarity_score": 1 - distance}  # Convert distance to similarity score
        for result, distance in zip(flattened_metadatas, flattened_distances)
        if "Organization Name" in result
    ]
    
    # Sort results from most similar to least similar
    startup_results.sort(key=lambda x: x["similarity_score"], reverse=True)
    
    return startup_results

# Function to retrieve all fields from SQLite for a list of startup names
def fetch_startup_details_from_sqlite(startup_names):
    # Connect to the SQLite database
    conn = sqlite3.connect('ai_investments.db')
    cursor = conn.cursor()
    
    # Prepare a list to store the startup details
    all_startup_details = []
    
    # Query for each startup name
    for name in startup_names:
        query = "SELECT * FROM investments WHERE `Organization Name` = ?"
        cursor.execute(query, (name,))
        rows = cursor.fetchall()
        
        # If results are found, add them to the list
        for row in rows:
            all_startup_details.append(row)
    
    # Close the database connection
    conn.close()
    
    return all_startup_details

# Main function to search, retrieve details, and save as CSV
def search_and_export_to_csv(query_text, top_n=100, output_dir="/Users/ozgurguler/Downloads"):
    # Search for similar startups
    similar_startups = search_similar_startups(query_text, top_n=top_n)
    
    # Extract just the names from the search results
    startup_names = [startup["startup_name"] for startup in similar_startups]
    
    # Fetch details from SQLite for the similar startups
    startup_details = fetch_startup_details_from_sqlite(startup_names)
    
    # Define column names (adjust according to your database schema)
    column_names = ["Organization Name", "Last Funding Type", "Last Funding Date", "Last Funding Amount (in USD)",
                    "Headquarters Location", "Description", "Top 5 Investors", "Founders", "Most Recent Valuation Range",
                    "Founded Date", "Twitter", "LinkedIn", "Full Description", "Website"]
    
    # Create the output file path based on the query text
    output_file = os.path.join(output_dir, f"{query_text}_startups.csv")
    
    # Save the results to a CSV file
    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(column_names)  # Write header
        writer.writerows(startup_details)  # Write rows
    
    print(f"Results saved to {output_file}")

# Example usage: Search for a similar startup based on a query and export results
query = "retail e-commerce"
search_and_export_to_csv(query, top_n=100)
